In [1]:
import sys
sys.path.append("/home/leosmigel/Development/alphai/")

# Scot1land Slingshot Strategy

- Universe of Stocks: (All or QQQ)
- Filter: Stocks above their 50sma that have pulled back at least 5%
- Entry: Close above 4ema high
- Exit Low of Day


## 1. Get Imports

In [2]:
import datetime as dt
import numpy as np
import pandas as pd

from src.models.api import get_tickers, get_minute_bars

## 2. Set Backtest Timeframe

In [3]:
start = '2020-01-01'
end = '2021-12-31'

## 3. Create Universe

We'll web scrape QQQ for demonstration purposes. Keep in mind there's potential survivorship bias as this is a current shapshot and doesn't contain any tickers removed from QQQ over the backtesting timeperiod. We would want to iterate through the start and end dates above over the daily holdings to determine if a stock was helds in QQQ.

In [4]:
import csv
import requests
URL = "https://www.invesco.com/us/financial-products/etfs/holdings/main/holdings/0?audienceType=Investor&action=download&ticker=QQQ"

with requests.Session() as s:
    download = s.get(URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    components_list = list(cr)
df = pd.DataFrame(components_list)
header = df.iloc[0]
df = df.iloc[1:]
df.columns = header
df

,Fund Ticker,Security Identifier,Holding Ticker,Shares/Par Value,MarketValue,Weight,Name,Class of Shares,Sector,Date
1,QQQ,037833100,AAPL,"141,639,282","25,779,765,716.82",11.806,Apple Inc,Common Stock,Information Technology,01/03/2022
2,QQQ,594918104,MSFT,"64,817,701","21,697,725,409.75",9.937,Microsoft Corp,Common Stock,Information Technology,01/03/2022
3,QQQ,023135106,AMZN,"4,378,294","14,921,619,998.46",6.834,Amazon.com Inc,Common Stock,Consumer Discretionary,01/03/2022
4,QQQ,88160R101,TSLA,"8,669,992","10,402,083,001.76",4.764,Tesla Inc,Common Stock,Consumer Discretionary,01/03/2022
5,QQQ,30303M102,FB,"30,589,503","10,355,770,345.62",4.743,Meta Platforms Inc,Common Stock,Communication Services,01/03/2022
...,...,...,...,...,...,...,...,...,...,...
97,QQQ,83088M102,SWKS,"2,368,847","378,494,373.66",0.173,Skyworks Solutions Inc,Common Stock,Information Technology,01/03/2022
98,QQQ,82968B103,SIRI,"57,284,943","364,332,237.48",0.167,Sirius XM Holdings Inc,Common Stock,Communication Services,01/03/2022
99,QQQ,722304102,PDD,"5,954,777","334,062,989.70",0.153,Pinduoduo Inc ADR,American Depository Receipt,Consumer Discretionary,01/03/2022
100,QQQ,848637104,SPLK,"2,313,829","270,625,439.84",0.124,Splunk Inc,Common Stock,Information Technology,01/03/2022


In [5]:
tickers = df['Holding Ticker'].str.strip().sort_values().to_list()
tickers



['AAPL',
 'ABNB',
 'ADBE',
 'ADI',
 'ADP',
 'ADSK',
 'AEP',
 'ALGN',
 'AMAT',
 'AMD',
 'AMGN',
 'AMZN',
 'ANSS',
 'ASML',
 'ATVI',
 'AVGO',
 'BIDU',
 'BIIB',
 'BKNG',
 'CDNS',
 'CHTR',
 'CMCSA',
 'COST',
 'CPRT',
 'CRWD',
 'CSCO',
 'CSX',
 'CTAS',
 'CTSH',
 'DDOG',
 'DLTR',
 'DOCU',
 'DXCM',
 'EA',
 'EBAY',
 'EXC',
 'FAST',
 'FB',
 'FISV',
 'FTNT',
 'GILD',
 'GOOG',
 'GOOGL',
 'HON',
 'IDXX',
 'ILMN',
 'INTC',
 'INTU',
 'ISRG',
 'JD',
 'KDP',
 'KHC',
 'KLAC',
 'LCID',
 'LRCX',
 'LULU',
 'MAR',
 'MCHP',
 'MDLZ',
 'MELI',
 'MNST',
 'MRNA',
 'MRVL',
 'MSFT',
 'MTCH',
 'MU',
 'NFLX',
 'NTES',
 'NVDA',
 'NXPI',
 'OKTA',
 'ORLY',
 'PANW',
 'PAYX',
 'PCAR',
 'PDD',
 'PEP',
 'PTON',
 'PYPL',
 'QCOM',
 'REGN',
 'ROST',
 'SBUX',
 'SGEN',
 'SIRI',
 'SNPS',
 'SPLK',
 'SWKS',
 'TEAM',
 'TMUS',
 'TSLA',
 'TXN',
 'VRSK',
 'VRSN',
 'VRTX',
 'WBA',
 'WDAY',
 'XEL',
 'XLNX',
 'ZM',
 'ZS']

In [6]:
#tickers = get_tickers(market='stock')
#tickers = tickers[:300]

## 4. Get Bar Data


In [7]:
bars1m = get_minute_bars(tickers, start, end)
bars1m

2020-01-01
2021-12-31


open      high       low     close  \
ticker date                                                                
AAPL   2020-01-02 14:30:00+00:00   74.0600   74.0750   73.8475   73.9387   
       2020-01-02 14:31:00+00:00   73.9250   73.9775   73.8259   73.8825   
       2020-01-02 14:32:00+00:00   73.8725   73.9525   73.7975   73.9263   
       2020-01-02 14:33:00+00:00   73.9275   74.2000   73.9250   74.1375   
       2020-01-02 14:34:00+00:00   74.1325   74.2315   74.0300   74.2175   
...                                    ...       ...       ...       ...   
ZS     2021-12-31 20:55:00+00:00  324.4000  324.6300  324.1800  324.3700   
       2021-12-31 20:56:00+00:00  324.4000  324.6300  324.1800  324.3700   
       2021-12-31 20:57:00+00:00  324.4000  324.6300  324.1800  324.3700   
       2021-12-31 20:58:00+00:00  324.4000  324.6300  324.1800  324.3700   
       2021-12-31 20:59:00+00:00  324.4000  324.6300  324.1800  324.3700   

                                     volume  
ticker date                                  
AAPL   2020-01-02 14:30:00+00:00  3407884.0  
       2020-01-02 14:31:00+00:00   921140.0  
       2020-01-02 14:32:00+00:00   802780.0  
       2020-01-02 14:33:00+00:00  1324276.0  
       2020-01-02 14:34:00+00:00   911928.0  
...                                     ...  
ZS     2021-12-31 20:55:00+00:00        0.0  
       2021-12-31 20:56:00+00:00        0.0  
       2021-12-31 20:57:00+00:00        0.0  
       2021-12-31 20:58:00+00:00        0.0  
       2021-12-31 20:59:00+00:00        0.0  

[19837410 rows x 5 columns]

In [8]:
bars1m = bars1m.reset_index()

In [9]:
bars1m[bars1m['date'] < '2021-03-14'] # Sunday

,ticker,date,open,high,low,close,volume
0,AAPL,2020-01-02 14:30:00+00:00,74.0600,74.0750,73.8475,73.9387,3407884.0
1,AAPL,2020-01-02 14:31:00+00:00,73.9250,73.9775,73.8259,73.8825,921140.0
2,AAPL,2020-01-02 14:32:00+00:00,73.8725,73.9525,73.7975,73.9263,802780.0
3,AAPL,2020-01-02 14:33:00+00:00,73.9275,74.2000,73.9250,74.1375,1324276.0
4,AAPL,2020-01-02 14:34:00+00:00,74.1325,74.2315,74.0300,74.2175,911928.0
...,...,...,...,...,...,...,...
19758025,ZS,2021-03-12 20:55:00+00:00,184.2300,185.0199,184.2300,184.8900,14634.0
19758026,ZS,2021-03-12 20:56:00+00:00,184.9000,184.9000,184.5200,184.6000,8669.0
19758027,ZS,2021-03-12 20:57:00+00:00,184.6000,184.6100,184.3600,184.4600,12071.0
19758028,ZS,2021-03-12 20:58:00+00:00,184.4883,184.5000,184.3200,184.4100,13066.0


In [10]:
bars1m[bars1m['date'] >= '2021-03-14'] # Sunday


,ticker,date,open,high,low,close,volume
117030,AAPL,2021-03-15 13:30:00+00:00,121.41,121.500,121.2600,121.26,1193977.0
117031,AAPL,2021-03-15 13:31:00+00:00,121.26,121.280,121.1001,121.19,367681.0
117032,AAPL,2021-03-15 13:32:00+00:00,121.19,121.205,120.8400,120.84,645914.0
117033,AAPL,2021-03-15 13:33:00+00:00,120.85,120.920,120.4200,120.77,567500.0
117034,AAPL,2021-03-15 13:34:00+00:00,120.78,121.000,120.7500,120.89,369702.0
...,...,...,...,...,...,...,...
19837405,ZS,2021-12-31 20:55:00+00:00,324.40,324.630,324.1800,324.37,0.0
19837406,ZS,2021-12-31 20:56:00+00:00,324.40,324.630,324.1800,324.37,0.0
19837407,ZS,2021-12-31 20:57:00+00:00,324.40,324.630,324.1800,324.37,0.0
19837408,ZS,2021-12-31 20:58:00+00:00,324.40,324.630,324.1800,324.37,0.0


In [11]:
bars1m[bars1m['date'] < '2021-11-07'] # Sunday

,ticker,date,open,high,low,close,volume
0,AAPL,2020-01-02 14:30:00+00:00,74.0600,74.0750,73.8475,73.9387,3407884.0
1,AAPL,2020-01-02 14:31:00+00:00,73.9250,73.9775,73.8259,73.8825,921140.0
2,AAPL,2020-01-02 14:32:00+00:00,73.8725,73.9525,73.7975,73.9263,802780.0
3,AAPL,2020-01-02 14:33:00+00:00,73.9275,74.2000,73.9250,74.1375,1324276.0
4,AAPL,2020-01-02 14:34:00+00:00,74.1325,74.2315,74.0300,74.2175,911928.0
...,...,...,...,...,...,...,...
19822765,ZS,2021-11-05 19:55:00+00:00,337.9300,338.2500,337.9000,338.0250,9863.0
19822766,ZS,2021-11-05 19:56:00+00:00,338.0100,338.2300,337.9000,337.9850,15010.0
19822767,ZS,2021-11-05 19:57:00+00:00,337.9850,338.1800,337.9850,338.1050,13638.0
19822768,ZS,2021-11-05 19:58:00+00:00,338.2310,338.2600,338.0800,338.1800,17377.0


In [12]:
bars1m['date'] = bars1m['date'].dt.tz_convert('America/New_York')
#bars1m = bars1m.set_index(['ticker','date'])
bars1m

,ticker,date,open,high,low,close,volume
0,AAPL,2020-01-02 09:30:00-05:00,74.0600,74.0750,73.8475,73.9387,3407884.0
1,AAPL,2020-01-02 09:31:00-05:00,73.9250,73.9775,73.8259,73.8825,921140.0
2,AAPL,2020-01-02 09:32:00-05:00,73.8725,73.9525,73.7975,73.9263,802780.0
3,AAPL,2020-01-02 09:33:00-05:00,73.9275,74.2000,73.9250,74.1375,1324276.0
4,AAPL,2020-01-02 09:34:00-05:00,74.1325,74.2315,74.0300,74.2175,911928.0
...,...,...,...,...,...,...,...
19837405,ZS,2021-12-31 15:55:00-05:00,324.4000,324.6300,324.1800,324.3700,0.0
19837406,ZS,2021-12-31 15:56:00-05:00,324.4000,324.6300,324.1800,324.3700,0.0
19837407,ZS,2021-12-31 15:57:00-05:00,324.4000,324.6300,324.1800,324.3700,0.0
19837408,ZS,2021-12-31 15:58:00-05:00,324.4000,324.6300,324.1800,324.3700,0.0


In [13]:
bars1m[bars1m['date'] >= '2021-03-14'] # Sunday Begin

,ticker,date,open,high,low,close,volume
117030,AAPL,2021-03-15 09:30:00-04:00,121.41,121.500,121.2600,121.26,1193977.0
117031,AAPL,2021-03-15 09:31:00-04:00,121.26,121.280,121.1001,121.19,367681.0
117032,AAPL,2021-03-15 09:32:00-04:00,121.19,121.205,120.8400,120.84,645914.0
117033,AAPL,2021-03-15 09:33:00-04:00,120.85,120.920,120.4200,120.77,567500.0
117034,AAPL,2021-03-15 09:34:00-04:00,120.78,121.000,120.7500,120.89,369702.0
...,...,...,...,...,...,...,...
19837405,ZS,2021-12-31 15:55:00-05:00,324.40,324.630,324.1800,324.37,0.0
19837406,ZS,2021-12-31 15:56:00-05:00,324.40,324.630,324.1800,324.37,0.0
19837407,ZS,2021-12-31 15:57:00-05:00,324.40,324.630,324.1800,324.37,0.0
19837408,ZS,2021-12-31 15:58:00-05:00,324.40,324.630,324.1800,324.37,0.0


In [14]:
#bars1m = bars1m.reset_index().set_index(['date','ticker'])
bars1m = bars1m.set_index(['date','ticker'])

bars1m

,,open,high,low,close,volume
date,ticker,,,,,
2020-01-02 09:30:00-05:00,AAPL,74.0600,74.0750,73.8475,73.9387,3407884.0
2020-01-02 09:31:00-05:00,AAPL,73.9250,73.9775,73.8259,73.8825,921140.0
2020-01-02 09:32:00-05:00,AAPL,73.8725,73.9525,73.7975,73.9263,802780.0
2020-01-02 09:33:00-05:00,AAPL,73.9275,74.2000,73.9250,74.1375,1324276.0
2020-01-02 09:34:00-05:00,AAPL,74.1325,74.2315,74.0300,74.2175,911928.0
...,...,...,...,...,...,...
2021-12-31 15:55:00-05:00,ZS,324.4000,324.6300,324.1800,324.3700,0.0
2021-12-31 15:56:00-05:00,ZS,324.4000,324.6300,324.1800,324.3700,0.0
2021-12-31 15:57:00-05:00,ZS,324.4000,324.6300,324.1800,324.3700,0.0


## 5. Create Indicators

It's easiest to create the indicators first so you can apply it over multiple timeframes.

In [15]:
ema4 = lambda x: x.ewm(span=4, min_periods=4, adjust=False, ignore_na=False).mean()
ema10 = lambda x: x.ewm(span=10, min_periods=10, adjust=False, ignore_na=False).mean()
sma50 = lambda x: x.rolling(50).mean()

## 6. Resample Timeframes

In [16]:
aggdict = {
    'open':'first',
    'high':'max',
    'low':'min',
    'close':'last',
    'volume':'sum'
}

In [17]:
bars1h = bars1m.groupby([pd.Grouper(level=0, freq='1h', label='left'),
                         pd.Grouper(level=1)]).agg(aggdict)
bars1h['ema4'] = bars1h.groupby('ticker')['high'].apply(ema4)
bars1h = bars1h.dropna(subset=['ema4'])
bars1h.columns = [c + '_1h' for c in bars1h.columns]
bars1h

open_1h   high_1h    low_1h  close_1h  \
date                      ticker                                           
2020-01-02 12:00:00-05:00 AAPL     74.6025   74.7000   74.5125   74.6752   
                          ADBE    333.1200  333.3143  332.4700  332.9200   
                          ADI     119.4800  119.8200  119.4100  119.7200   
                          ADP     169.6800  170.1900  169.5800  170.0855   
                          ADSK    186.0600  186.6000  185.7600  186.4200   
...                                    ...       ...       ...       ...   
2021-12-31 15:00:00-05:00 WDAY    276.5400  276.7700  276.4200  276.4900   
                          XEL      68.2650   68.2800   68.2300   68.2300   
                          XLNX    213.9200  214.2100  213.3800  213.8900   
                          ZM      190.6950  190.7200  190.4600  190.5000   
                          ZS      324.4000  324.6300  324.1800  324.3700   

                                   volume_1h     ema4_1h  
date                      ticker                          
2020-01-02 12:00:00-05:00 AAPL    10129932.0   74.647860  
                          ADBE      135720.0  333.411080  
                          ADI        73220.0  119.895840  
                          ADP        98475.0  170.703840  
                          ADSK       75039.0  186.839640  
...                                      ...         ...  
2021-12-31 15:00:00-05:00 WDAY           0.0  276.832598  
                          XEL            0.0   68.277573  
                          XLNX           0.0  214.298684  
                          ZM             0.0  190.753506  
                          ZS             0.0  324.739714  

[351411 rows x 6 columns]

In [18]:
bars1h.isnull().sum()

open_1h      0
high_1h      0
low_1h       0
close_1h     0
volume_1h    0
ema4_1h      0
dtype: int64

In [19]:
bars1d = bars1m.groupby([pd.Grouper(level=0, freq='1d', label='left'),
                         pd.Grouper(level=1)]).agg(aggdict)
bars1d['ema4'] = bars1d.groupby('ticker')['high'].apply(ema4)
bars1d['ema10'] = bars1d.groupby('ticker')['close'].apply(ema10)
bars1d['sma50'] = bars1d.groupby('ticker')['close'].apply(sma50)
bars1d = bars1d.dropna(subset=['sma50'])
bars1d.columns = [c + '_1d' for c in bars1d.columns]
bars1d

open_1d  high_1d    low_1d  close_1d  \
date                      ticker                                          
2020-03-13 00:00:00-04:00 AAPL     66.2225    69.98   63.2375    69.515   
                          ADBE    312.8800   336.00  298.0001   335.500   
                          ADI      94.8800   100.46   89.3600    99.950   
                          ADP     136.9900   144.53  129.2700   144.500   
                          ADSK    146.0000   158.17  138.4500   158.030   
...                                    ...      ...       ...       ...   
2021-12-31 00:00:00-05:00 WDAY    276.5400   276.77  276.4200   276.490   
                          XEL      68.2650    68.28   68.2300    68.230   
                          XLNX    213.9200   214.21  213.3800   213.890   
                          ZM      190.6950   190.72  190.4600   190.500   
                          ZS      324.4000   324.63  324.1800   324.370   

                                    volume_1d     ema4_1d    ema10_1d  \
date                      ticker                                        
2020-03-13 00:00:00-04:00 AAPL    327881760.0   69.868001   69.796870   
                          ADBE      8023651.0  328.868196  329.088152   
                          ADI       5376881.0  100.388313  101.947219   
                          ADP       3022610.0  147.038180  151.225610   
                          ADSK      3677517.0  159.680186  165.738526   
...                                       ...         ...         ...   
2021-12-31 00:00:00-05:00 WDAY            0.0  278.094047  276.524339   
                          XEL             0.0   68.049350   67.528974   
                          XLNX            0.0  218.073918  214.500995   
                          ZM              0.0  190.919280  189.525046   
                          ZS              0.0  326.768251  320.479219   

                                    sma50_1d  
date                      ticker              
2020-03-13 00:00:00-04:00 AAPL     76.458082  
                          ADBE    350.494600  
                          ADI     114.137600  
                          ADP     169.915600  
                          ADSK    191.567300  
...                                      ...  
2021-12-31 00:00:00-05:00 WDAY    282.006800  
                          XEL      65.503400  
                          XLNX    207.772700  
                          ZM      227.417700  
                          ZS      325.350900  

[45425 rows x 8 columns]

In [20]:
bars1d.isnull().sum()

open_1d      0
high_1d      0
low_1d       0
close_1d     0
volume_1d    0
ema4_1d      0
ema10_1d     0
sma50_1d     0
dtype: int64

In [21]:
bars1d = bars1d.reset_index()
bars1d['date'] = bars1d['date'] + pd.DateOffset(hours=15)
bars1d = bars1d.set_index(['date','ticker'])
bars1d

open_1d  high_1d    low_1d  close_1d  \
date                      ticker                                          
2020-03-13 15:00:00-04:00 AAPL     66.2225    69.98   63.2375    69.515   
                          ADBE    312.8800   336.00  298.0001   335.500   
                          ADI      94.8800   100.46   89.3600    99.950   
                          ADP     136.9900   144.53  129.2700   144.500   
                          ADSK    146.0000   158.17  138.4500   158.030   
...                                    ...      ...       ...       ...   
2021-12-31 15:00:00-05:00 WDAY    276.5400   276.77  276.4200   276.490   
                          XEL      68.2650    68.28   68.2300    68.230   
                          XLNX    213.9200   214.21  213.3800   213.890   
                          ZM      190.6950   190.72  190.4600   190.500   
                          ZS      324.4000   324.63  324.1800   324.370   

                                    volume_1d     ema4_1d    ema10_1d  \
date                      ticker                                        
2020-03-13 15:00:00-04:00 AAPL    327881760.0   69.868001   69.796870   
                          ADBE      8023651.0  328.868196  329.088152   
                          ADI       5376881.0  100.388313  101.947219   
                          ADP       3022610.0  147.038180  151.225610   
                          ADSK      3677517.0  159.680186  165.738526   
...                                       ...         ...         ...   
2021-12-31 15:00:00-05:00 WDAY            0.0  278.094047  276.524339   
                          XEL             0.0   68.049350   67.528974   
                          XLNX            0.0  218.073918  214.500995   
                          ZM              0.0  190.919280  189.525046   
                          ZS              0.0  326.768251  320.479219   

                                    sma50_1d  
date                      ticker              
2020-03-13 15:00:00-04:00 AAPL     76.458082  
                          ADBE    350.494600  
                          ADI     114.137600  
                          ADP     169.915600  
                          ADSK    191.567300  
...                                      ...  
2021-12-31 15:00:00-05:00 WDAY    282.006800  
                          XEL      65.503400  
                          XLNX    207.772700  
                          ZM      227.417700  
                          ZS      325.350900  

[45425 rows x 8 columns]

## 7. Merge Timeframes

In [22]:
print(bars1d.index[0][0])
print(bars1h.index[0][0])
print(bars1d.index[-1][0])
print(bars1h.index[-1][0])

2020-03-13 15:00:00-04:00
2020-01-02 12:00:00-05:00
2021-12-31 15:00:00-05:00
2021-12-31 15:00:00-05:00


In [23]:
bars1h = bars1h.reset_index()
bars1d = bars1d.reset_index()

In [24]:
#bars1d = bars1d.loc[bars1d['date'] >= bars1h['date'].min(), :]
bars1h = bars1h.loc[bars1h['date'] >= bars1d['date'].min(), :]
#bars1d = bars1d.loc[bars1d['date'] <= bars1h['date'].max(), :]
#bars1h = bars1h.loc[bars1h['date'] <= bars1d['date'].max(), :]

In [25]:
print(bars1d.date.iloc[0])
print(bars1h.date.iloc[0])
print(bars1d.date.iloc[-1])
print(bars1h.date.iloc[-1])

2020-03-13 15:00:00-04:00
2020-03-13 15:00:00-04:00
2021-12-31 15:00:00-05:00
2021-12-31 15:00:00-05:00


In [26]:
bars1h[bars1h.date >= '2021-03-16']

,date,ticker,open_1h,high_1h,low_1h,close_1h,volume_1h,ema4_1h
208833,2021-03-16 09:00:00-04:00,AAPL,125.7000,126.6400,125.0800,126.520,18398047.0,124.772036
208834,2021-03-16 09:00:00-04:00,ABNB,209.1100,215.4900,208.1600,211.525,875759.0,212.728501
208835,2021-03-16 09:00:00-04:00,ADBE,452.0100,455.4804,451.8700,455.230,262218.0,449.827763
208836,2021-03-16 09:00:00-04:00,ADI,152.3374,153.4800,151.9400,153.400,225426.0,151.620766
208837,2021-03-16 09:00:00-04:00,ADP,188.7500,190.5000,188.0416,189.040,197075.0,188.887539
...,...,...,...,...,...,...,...,...
351406,2021-12-31 15:00:00-05:00,WDAY,276.5400,276.7700,276.4200,276.490,0.0,276.832598
351407,2021-12-31 15:00:00-05:00,XEL,68.2650,68.2800,68.2300,68.230,0.0,68.277573
351408,2021-12-31 15:00:00-05:00,XLNX,213.9200,214.2100,213.3800,213.890,0.0,214.298684
351409,2021-12-31 15:00:00-05:00,ZM,190.6950,190.7200,190.4600,190.500,0.0,190.753506


In [27]:
bars = bars1h.merge(bars1d, on=['date','ticker'], how='left')
bars[bars1d.columns] = bars.groupby('ticker')[bars1d.columns].transform(lambda x: x.ffill())
bars = bars.set_index(['date','ticker'])
bars

open_1h  high_1h  low_1h  close_1h  \
date                      ticker                                        
2020-03-13 15:00:00-04:00 AAPL     64.8000    69.98   63.59    69.515   
                          ADBE    307.6400   336.00  303.45   335.500   
                          ADI      93.0800   100.05   93.00    99.950   
                          ADP     134.2256   144.53  130.07   144.500   
                          ADSK    147.4200   158.17  146.21   158.030   
...                                    ...      ...     ...       ...   
2021-12-31 15:00:00-05:00 WDAY    276.5400   276.77  276.42   276.490   
                          XEL      68.2650    68.28   68.23    68.230   
                          XLNX    213.9200   214.21  213.38   213.890   
                          ZM      190.6950   190.72  190.46   190.500   
                          ZS      324.4000   324.63  324.18   324.370   

                                   volume_1h     ema4_1h   open_1d  high_1d  \
date                      ticker                                              
2020-03-13 15:00:00-04:00 AAPL    75751500.0   67.440044   66.2225    69.98   
                          ADBE     2434752.0  320.421871  312.8800   336.00   
                          ADI      1641886.0   96.647788   94.8800   100.46   
                          ADP      1015457.0  139.083862  136.9900   144.53   
                          ADSK      973331.0  152.935020  146.0000   158.17   
...                                      ...         ...       ...      ...   
2021-12-31 15:00:00-05:00 WDAY           0.0  276.832598  276.5400   276.77   
                          XEL            0.0   68.277573   68.2650    68.28   
                          XLNX           0.0  214.298684  213.9200   214.21   
                          ZM             0.0  190.753506  190.6950   190.72   
                          ZS             0.0  324.739714  324.4000   324.63   

                                    low_1d  close_1d    volume_1d     ema4_1d  \
date                      ticker                                                
2020-03-13 15:00:00-04:00 AAPL     63.2375    69.515  327881760.0   69.868001   
                          ADBE    298.0001   335.500    8023651.0  328.868196   
                          ADI      89.3600    99.950    5376881.0  100.388313   
                          ADP     129.2700   144.500    3022610.0  147.038180   
                          ADSK    138.4500   158.030    3677517.0  159.680186   
...                                    ...       ...          ...         ...   
2021-12-31 15:00:00-05:00 WDAY    276.4200   276.490          0.0  278.094047   
                          XEL      68.2300    68.230          0.0   68.049350   
                          XLNX    213.3800   213.890          0.0  218.073918   
                          ZM      190.4600   190.500          0.0  190.919280   
                          ZS      324.1800   324.370          0.0  326.768251   

                                    ema10_1d    sma50_1d  
date                      ticker                          
2020-03-13 15:00:00-04:00 AAPL     69.796870   76.458082  
                          ADBE    329.088152  350.494600  
                          ADI     101.947219  114.137600  
                          ADP     151.225610  169.915600  
                          ADSK    165.738526  191.567300  
...                                      ...         ...  
2021-12-31 15:00:00-05:00 WDAY    276.524339  282.006800  
                          XEL      67.528974   65.503400  
                          XLNX    214.500995  207.772700  
                          ZM      189.525046  227.417700  
                          ZS      320.479219  325.350900  

[317157 rows x 14 columns]

In [28]:
bars

open_1h  high_1h  low_1h  close_1h  \
date                      ticker                                        
2020-03-13 15:00:00-04:00 AAPL     64.8000    69.98   63.59    69.515   
                          ADBE    307.6400   336.00  303.45   335.500   
                          ADI      93.0800   100.05   93.00    99.950   
                          ADP     134.2256   144.53  130.07   144.500   
                          ADSK    147.4200   158.17  146.21   158.030   
...                                    ...      ...     ...       ...   
2021-12-31 15:00:00-05:00 WDAY    276.5400   276.77  276.42   276.490   
                          XEL      68.2650    68.28   68.23    68.230   
                          XLNX    213.9200   214.21  213.38   213.890   
                          ZM      190.6950   190.72  190.46   190.500   
                          ZS      324.4000   324.63  324.18   324.370   

                                   volume_1h     ema4_1h   open_1d  high_1d  \
date                      ticker                                              
2020-03-13 15:00:00-04:00 AAPL    75751500.0   67.440044   66.2225    69.98   
                          ADBE     2434752.0  320.421871  312.8800   336.00   
                          ADI      1641886.0   96.647788   94.8800   100.46   
                          ADP      1015457.0  139.083862  136.9900   144.53   
                          ADSK      973331.0  152.935020  146.0000   158.17   
...                                      ...         ...       ...      ...   
2021-12-31 15:00:00-05:00 WDAY           0.0  276.832598  276.5400   276.77   
                          XEL            0.0   68.277573   68.2650    68.28   
                          XLNX           0.0  214.298684  213.9200   214.21   
                          ZM             0.0  190.753506  190.6950   190.72   
                          ZS             0.0  324.739714  324.4000   324.63   

                                    low_1d  close_1d    volume_1d     ema4_1d  \
date                      ticker                                                
2020-03-13 15:00:00-04:00 AAPL     63.2375    69.515  327881760.0   69.868001   
                          ADBE    298.0001   335.500    8023651.0  328.868196   
                          ADI      89.3600    99.950    5376881.0  100.388313   
                          ADP     129.2700   144.500    3022610.0  147.038180   
                          ADSK    138.4500   158.030    3677517.0  159.680186   
...                                    ...       ...          ...         ...   
2021-12-31 15:00:00-05:00 WDAY    276.4200   276.490          0.0  278.094047   
                          XEL      68.2300    68.230          0.0   68.049350   
                          XLNX    213.3800   213.890          0.0  218.073918   
                          ZM      190.4600   190.500          0.0  190.919280   
                          ZS      324.1800   324.370          0.0  326.768251   

                                    ema10_1d    sma50_1d  
date                      ticker                          
2020-03-13 15:00:00-04:00 AAPL     69.796870   76.458082  
                          ADBE    329.088152  350.494600  
                          ADI     101.947219  114.137600  
                          ADP     151.225610  169.915600  
                          ADSK    165.738526  191.567300  
...                                      ...         ...  
2021-12-31 15:00:00-05:00 WDAY    276.524339  282.006800  
                          XEL      67.528974   65.503400  
                          XLNX    214.500995  207.772700  
                          ZM      189.525046  227.417700  
                          ZS      320.479219  325.350900  

[317157 rows x 14 columns]

In [29]:
bars.swaplevel(1,0).xs('AAPL').isnull().sum()

open_1h      0
high_1h      0
low_1h       0
close_1h     0
volume_1h    0
ema4_1h      0
open_1d      0
high_1d      0
low_1d       0
close_1d     0
volume_1d    0
ema4_1d      0
ema10_1d     0
sma50_1d     0
dtype: int64

## 8. Create Signals

#### Entry Signal
If the stock is above the 50 day, and has suffered a recent downturn, but has now crossed the 4ema high, buy.

In [38]:
bars['entry_signal_h']  = (
    (bars['close_1d'] > bars['sma50_1d']) &
    (bars['close_1d'] < bars['ema10_1d']) &
    (bars['close_1h'] > bars['ema4_1h']) &
    (bars.groupby('ticker')['close_1h'].shift() < \
     bars.groupby('ticker')['ema4_1h'].shift())
    
)

bars['entry_signal_d']  = (
    (bars['close_1d'] > bars['sma50_1d']) &
    (bars['close_1d'] < bars['ema10_1d']) &
    (bars['close_1d'] > bars['ema4_1d']) &
    (bars.groupby('ticker')['close_1d'].shift() < \
     bars.groupby('ticker')['ema4_1d'].shift())
    
)
bars

open_1h  high_1h  low_1h  close_1h  \
date                      ticker                                        
2020-03-13 15:00:00-04:00 AAPL     64.8000    69.98   63.59    69.515   
                          ADBE    307.6400   336.00  303.45   335.500   
                          ADI      93.0800   100.05   93.00    99.950   
                          ADP     134.2256   144.53  130.07   144.500   
                          ADSK    147.4200   158.17  146.21   158.030   
...                                    ...      ...     ...       ...   
2021-12-31 15:00:00-05:00 WDAY    276.5400   276.77  276.42   276.490   
                          XEL      68.2650    68.28   68.23    68.230   
                          XLNX    213.9200   214.21  213.38   213.890   
                          ZM      190.6950   190.72  190.46   190.500   
                          ZS      324.4000   324.63  324.18   324.370   

                                   volume_1h     ema4_1h   open_1d  high_1d  \
date                      ticker                                              
2020-03-13 15:00:00-04:00 AAPL    75751500.0   67.440044   66.2225    69.98   
                          ADBE     2434752.0  320.421871  312.8800   336.00   
                          ADI      1641886.0   96.647788   94.8800   100.46   
                          ADP      1015457.0  139.083862  136.9900   144.53   
                          ADSK      973331.0  152.935020  146.0000   158.17   
...                                      ...         ...       ...      ...   
2021-12-31 15:00:00-05:00 WDAY           0.0  276.832598  276.5400   276.77   
                          XEL            0.0   68.277573   68.2650    68.28   
                          XLNX           0.0  214.298684  213.9200   214.21   
                          ZM             0.0  190.753506  190.6950   190.72   
                          ZS             0.0  324.739714  324.4000   324.63   

                                    low_1d  close_1d    volume_1d     ema4_1d  \
date                      ticker                                                
2020-03-13 15:00:00-04:00 AAPL     63.2375    69.515  327881760.0   69.868001   
                          ADBE    298.0001   335.500    8023651.0  328.868196   
                          ADI      89.3600    99.950    5376881.0  100.388313   
                          ADP     129.2700   144.500    3022610.0  147.038180   
                          ADSK    138.4500   158.030    3677517.0  159.680186   
...                                    ...       ...          ...         ...   
2021-12-31 15:00:00-05:00 WDAY    276.4200   276.490          0.0  278.094047   
                          XEL      68.2300    68.230          0.0   68.049350   
                          XLNX    213.3800   213.890          0.0  218.073918   
                          ZM      190.4600   190.500          0.0  190.919280   
                          ZS      324.1800   324.370          0.0  326.768251   

                                    ema10_1d    sma50_1d  entry_signal  \
date                      ticker                                         
2020-03-13 15:00:00-04:00 AAPL     69.796870   76.458082         False   
                          ADBE    329.088152  350.494600         False   
                          ADI     101.947219  114.137600         False   
                          ADP     151.225610  169.915600         False   
                          ADSK    165.738526  191.567300         False   
...                                      ...         ...           ...   
2021-12-31 15:00:00-05:00 WDAY    276.524339  282.006800         False   
                          XEL      67.528974   65.503400         False   
                          XLNX    214.500995  207.772700         False   
                          ZM      189.525046  227.417700         False   
                          ZS      320.479219  325.350900         False   

                                  entry_

In [39]:
bars[bars['entry_signal_h'] == True]

open_1h   high_1h  low_1h  close_1h  \
date                      ticker                                         
2020-03-16 09:00:00-04:00 ZM      106.4900  109.3500  103.20  109.1550   
2020-03-16 10:00:00-04:00 GILD     67.5300   70.6400   66.61   70.3800   
2020-03-16 14:00:00-04:00 GILD     70.4600   72.3270   70.29   72.1501   
2020-03-17 10:00:00-04:00 GILD     70.1450   73.7500   69.94   73.6900   
                          REGN    479.1400  500.7465  476.69  499.4900   
...                                    ...       ...     ...       ...   
2021-12-27 15:00:00-05:00 XEL      66.6219   66.8500   66.53   66.8200   
2021-12-28 09:00:00-05:00 FISV    104.0000  104.8800  103.11  104.5400   
2021-12-28 11:00:00-05:00 FISV    104.7900  105.3700  104.54  105.1800   
2021-12-30 09:00:00-05:00 SIRI      6.3700    6.4400    6.35    6.4350   
2021-12-30 13:00:00-05:00 SIRI      6.4350    6.4600    6.43    6.4600   

                                  volume_1h     ema4_1h  open_1d  high_1d  \
date                      ticker                                            
2020-03-16 09:00:00-04:00 ZM      1161617.0  108.569393  113.000  113.800   
2020-03-16 10:00:00-04:00 GILD    4102733.0   70.003837   71.930   72.000   
2020-03-16 14:00:00-04:00 GILD    2516898.0   71.320865   71.930   72.000   
2020-03-17 10:00:00-04:00 GILD    5216428.0   72.536507   67.240   72.800   
                          REGN     474789.0  489.975950  452.510  484.580   
...                                     ...         ...      ...      ...   
2021-12-27 15:00:00-05:00 XEL      346128.0   66.730419   66.820   66.935   
2021-12-28 09:00:00-05:00 FISV     226967.0  104.393788  105.290  105.290   
2021-12-28 11:00:00-05:00 FISV     286203.0  105.030468  105.290  105.290   
2021-12-30 09:00:00-05:00 SIRI    1743660.0    6.409759    6.405    6.440   
2021-12-30 13:00:00-05:00 SIRI     926465.0    6.446481    6.405    6.440   

                                  low_1d  close_1d   volume_1d     ema4_1d  \
date                      ticker                                             
2020-03-16 09:00:00-04:00 ZM      104.16    107.50   8813662.0  114.663417   
2020-03-16 10:00:00-04:00 GILD     67.50     70.75  26934342.0   73.775033   
2020-03-16 14:00:00-04:00 GILD     67.50     70.75  26934342.0   73.775033   
2020-03-17 10:00:00-04:00 GILD     64.41     68.90  21348420.0   73.385020   
                          REGN    433.82    439.26   1734352.0  476.066135   
...                                  ...       ...         ...         ...   
2021-12-27 15:00:00-05:00 XEL      66.38     66.82    901606.0   67.142883   
2021-12-28 09:00:00-05:00 FISV    102.75    103.92   2568610.0  105.494373   
2021-12-28 11:00:00-05:00 FISV    102.75    103.92   2568610.0  105.494373   
2021-12-30 09:00:00-05:00 SIRI      6.36      6.37   9552025.0    6.463053   
2021-12-30 13:00:00-05:00 SIRI      6.36      6.37   9552025.0    6.463053   

                                    ema10_1d    sma50_1d  entry_signal  \
date                      ticker                                         
2020-03-16 09:00:00-04:00 ZM      110.505551   90.052296          True   
2020-03-16 10:00:00-04:00 GILD     72.486843   67.914500          True   
2020-03-16 14:00:00-04:00 GILD     72.486843   67.914500          True   
2020-03-17 10:00:00-04:00 GILD     71.834690   67.987700          True   
                          REGN    456.630876  401.905592          True   
...                                      ...         ...           ...   
2021-12-27 15:00:00-05:00 XEL      66.874536   65.285200          True   
2021-12-28 09:00:00-05:00 FISV    104.035584  102.582410          True   
2021-12-28 11:00:00-05:00 FISV    104.035584  102.582410          True   
2021-12-30 09:00:00-05:00 SIRI      6.378086    6.291800          True   
2021-12-30 13:00:00-05:00 SIRI      6.378086    6.291800          True   

                                  entry_signal_h  entry_signal_d  
date                     

In [40]:
bars.swaplevel(1,0).xs('GILD')['2020-03-16 13:00:00-04:00':]

,open_1h,high_1h,low_1h,close_1h,volume_1h,ema4_1h,open_1d,high_1d,low_1d,close_1d,volume_1d,ema4_1d,ema10_1d,sma50_1d,entry_signal,entry_signal_h,entry_signal_d
date,,,,,,,,,,,,,,,,,
2020-03-16 13:00:00-04:00,69.970,70.860,69.950,70.4600,1709394.0,70.650109,71.9300,72.00,67.500,70.75,26934342.0,73.775033,72.486843,67.9145,False,False,False
2020-03-16 14:00:00-04:00,70.460,72.327,70.290,72.1501,2516898.0,71.320865,71.9300,72.00,67.500,70.75,26934342.0,73.775033,72.486843,67.9145,True,True,False
2020-03-16 15:00:00-04:00,72.150,72.800,67.760,68.9000,5893771.0,71.912519,67.2400,72.80,64.410,68.90,21348420.0,73.385020,71.834690,67.9877,False,False,False
2020-03-17 09:00:00-04:00,70.000,71.450,69.000,70.1500,2533400.0,71.727511,67.2400,72.80,64.410,68.90,21348420.0,73.385020,71.834690,67.9877,False,False,False
2020-03-17 10:00:00-04:00,70.145,73.750,69.940,73.6900,5216428.0,72.536507,67.2400,72.80,64.410,68.90,21348420.0,73.385020,71.834690,67.9877,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 11:00:00-05:00,73.350,73.380,73.305,73.3600,0.0,73.422375,73.6635,74.12,73.260,73.36,2834583.0,73.615222,72.559458,69.1364,False,False,False
2021-12-31 12:00:00-05:00,73.350,73.380,73.305,73.3600,0.0,73.405425,73.6635,74.12,73.260,73.36,2834583.0,73.615222,72.559458,69.1364,False,False,False
2021-12-31 13:00:00-05:00,73.350,73.380,73.305,73.3600,0.0,73.395255,73.6635,74.12,73.260,73.36,2834583.0,73.615222,72.559458,69.1364,False,False,False


### 9. Analyze Results
Feel free to adjust the various entries and exit timeframes, which is why I didn't split out the hourly and daily since you can mix entries and exits.

In [33]:
rrr = [r for r in range(1,11)]
rrr

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [49]:
hourly = True
stats = {}
for j in rrr:
    
    trades = []
    max_won = 0
    max_lost = 0
    for ticker, data in bars.groupby(level=1):
        rr = j
        won = 0
        lost = 0
        cons_won = 0
        cons_lost = 0

        position = False
        take_position = False
        for i, r in data.iterrows():
            if take_position == True:
                entry_price = r['open_1h']
                entry_risk = entry_price - stop_price
                position = True
                take_position = False

            if position == True:
                if r['close_1h'] < stop_price:
                    lost += 1
                    cons_lost += 1
                    max_won = max_won if max_won > cons_won else cons_won
                    cons_won = 0

                    position = False
                elif r['close_1h'] > (entry_price + entry_risk * rr):
                    won += 1
                    cons_won += 1
                    max_lost = max_lost if max_lost > cons_lost else cons_lost
                    cons_lost = 0

                    position = False

            if r['entry_signal_h'] == True and position == False:
                take_position = True
                stop_price = r['low_1d']


        position = False
        max_won = max_won if max_won > cons_won else cons_won
        max_lost = max_lost if max_lost > cons_lost else cons_lost
        trades.append([ticker, won, lost])

    stats[j] = {}
    stats[j]['trade'] = trades
    stats[j]['win_streak'] = max_won
    stats[j]['lose_streak'] = max_lost
stats

{1: {'trade': [['AAPL', 12, 10],
   ['ABNB', 5, 7],
   ['ADBE', 17, 15],
   ['ADI', 14, 14],
   ['ADP', 16, 10],
   ['ADSK', 16, 6],
   ['AEP', 3, 10],
   ['ALGN', 25, 15],
   ['AMAT', 14, 19],
   ['AMD', 14, 16],
   ['AMGN', 6, 10],
   ['AMZN', 14, 9],
   ['ANSS', 28, 13],
   ['ASML', 20, 14],
   ['ATVI', 12, 4],
   ['AVGO', 22, 12],
   ['BIDU', 10, 9],
   ['BIIB', 5, 6],
   ['BKNG', 13, 14],
   ['CDNS', 16, 14],
   ['CHTR', 12, 8],
   ['CMCSA', 19, 13],
   ['COST', 18, 12],
   ['CPRT', 13, 15],
   ['CRWD', 16, 12],
   ['CSCO', 18, 19],
   ['CSX', 19, 19],
   ['CTAS', 19, 16],
   ['CTSH', 18, 17],
   ['DDOG', 13, 7],
   ['DLTR', 13, 10],
   ['DOCU', 12, 3],
   ['DXCM', 11, 5],
   ['EA', 14, 13],
   ['EBAY', 9, 11],
   ['EXC', 13, 16],
   ['FAST', 16, 7],
   ['FB', 15, 13],
   ['FISV', 7, 9],
   ['FTNT', 17, 12],
   ['GILD', 18, 8],
   ['GOOG', 24, 12],
   ['GOOGL', 26, 11],
   ['HON', 15, 15],
   ['IDXX', 18, 7],
   ['ILMN', 9, 5],
   ['INTC', 6, 9],
   ['INTU', 17, 18],
   ['ISRG', 1

In [50]:
data = pd.DataFrame()
for k, v in stats.items():
    df = pd.DataFrame(v['trade'])
    df['rr'] = k
    df['win_streak'] = v['win_streak']
    df['lose_streak'] = v['lose_streak']
    df.columns = ['ticker','won','lost', 'rr', 'win_streak','lose_streak']
    data = data.append(df)
data

,ticker,won,lost,rr,win_streak,lose_streak
0,AAPL,12,10,1,10,10
1,ABNB,5,7,1,10,10
2,ADBE,17,15,1,10,10
3,ADI,14,14,1,10,10
4,ADP,16,10,1,10,10
...,...,...,...,...,...,...
96,WDAY,3,3,10,4,14
97,XEL,0,6,10,4,14
98,XLNX,4,14,10,4,14
99,ZM,2,6,10,4,14


In [51]:
sumstats = data.groupby('rr').agg({'won':'sum',
                                   'lost':'sum',
                                   'win_streak':'first',
                                   'lose_streak':'last'})
sumstats['total'] = sumstats['won'] + sumstats['lost']
sumstats['win_perc'] = sumstats['won'] / sumstats['total']
sumstats = sumstats.reset_index()
sumstats

,rr,won,lost,win_streak,lose_streak,total,win_perc
0,1,1459,1191,10,10,2650,0.550566
1,2,894,1183,11,12,2077,0.430429
2,3,646,1145,8,13,1791,0.360692
3,4,502,1124,5,14,1626,0.308733
4,5,411,1058,4,14,1469,0.279782
5,6,347,996,4,16,1343,0.258377
6,7,295,949,4,15,1244,0.237138
7,8,260,903,4,14,1163,0.223560
8,9,233,863,4,16,1096,0.212591
9,10,206,823,4,14,1029,0.200194


In [52]:
sumstats['edge'] = sumstats['win_perc'] * sumstats['rr'] - (1 - sumstats['win_perc']) * 1
sumstats

,rr,won,lost,win_streak,lose_streak,total,win_perc,edge
0,1,1459,1191,10,10,2650,0.550566,0.101132
1,2,894,1183,11,12,2077,0.430429,0.291286
2,3,646,1145,8,13,1791,0.360692,0.442769
3,4,502,1124,5,14,1626,0.308733,0.543665
4,5,411,1058,4,14,1469,0.279782,0.678693
5,6,347,996,4,16,1343,0.258377,0.808637
6,7,295,949,4,15,1244,0.237138,0.897106
7,8,260,903,4,14,1163,0.223560,1.012038
8,9,233,863,4,16,1096,0.212591,1.125912
9,10,206,823,4,14,1029,0.200194,1.202138
